In [33]:
import pandas as pd
import numpy as np
import pickle
import json
from datetime import datetime
import json
from matplotlib import pyplot as plt

from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomTreesEmbedding
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LassoCV, RidgeCV

from sklearn.model_selection import validation_curve
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV


from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_log_error, make_scorer
from scipy.stats import randint as randint
from scipy.stats import randint as sp_randint

import seaborn as sns

can_buy - возможно ли купить этот товар,
can_promote - возможность продвигать товар,
category - категория товара,
contacts_visible - видны ли контакты,
date_created - таймстемп создания товара,
delivery_available - возможность доставки,
description - описание,
fields - свойства товара,
id - идентификатор продукта,
images - id картинок,
location - местрорасположение,
mortgage_available - возможность ипотеки,
name - навзаник,
payment_available - возможность онлайн оплаты,
price - цена (в копейках),
subcategory - подкатегория,
subway - метро,

Цена в КОПЕЙКАХ Но предсказания нужны в РУБЛЯХ, потому - сразу поделите в train прайс на 100

train_4_col_2.csv.zip - кусок TRAIN, который содержит id, category, 

train_4_col_3.csv.zip - кусок TRAIN, который содержит id, fields (строки все)

train_4_col.csv.zip - кусок TRAIN, который содержит id, name, description, price (строки все)

train_sample.pckl.zip - кусок TRAIN, который содержит все поля, но только первые 100000 строк

# Получаем данные

In [34]:
train_ics = pd.read_csv('train_4_col_2.csv', delimiter='\t')
train_if = pd.read_csv('train_4_col_3.csv', delimiter='\t')
train_indp = pd.read_csv('train_4_col.csv', delimiter='\t')

In [35]:
with open('train_sample.pckl', 'rb') as f:
    train_sample = pickle.load(f)

In [36]:
train_ics.T

,0,1,2,3,4,5,6,7,8,9,...,1748880,1748881,1748882,1748883,1748884,1748885,1748886,1748887,1748888,1748889
Unnamed: 0,0,1,2,3,5,6,7,8,10,11,...,99912,99914,99915,99918,99920,99922,99923,99926,99928,99930
id,3edeb34cf93f490ff760af85,c98febd50dad3cc0ffc86085,ade01e13912a46a99134cc75,ab3e6941c11304c1519aef75,252452a91c944a22c276d995,364f7659a4e9ba6c53e35495,c3d87065e4dbc080f400bb85,22e6a518d53f490f6ce7fd85,c6bd550b7f1304c1f7fda485,117b885a3cf9554042ae2285,...,97ee729c6b6b083a05e14995,3e37e6d8bc70bf66f2ca7495,8d403112ca9e2db5ce63d885,2ccac518d420532f256f4595,60ea05f603ff162bd4260195,2460206aa90c63db4475fe85,36f2465b7a5402fcdb8dd595,3cd221a6921fcb56e2dfcd85,40756707daff162bfe68d195,3cddf37632e395c29d764095
category,9,22,22,22,22,9,15,4,3,2,...,2,12,9,9,20,8,4,15,14,12
subcategory,914,2202,2204,2209,2204,902,1508,403,301,208,...,208,1202,904,921,2004,803,406,1507,1403,1202


In [37]:
train_if.T

,0,1,2,3,4,5,6,7,8,9,...,1748880,1748881,1748882,1748883,1748884,1748885,1748886,1748887,1748888,1748889
Unnamed: 0,0,1,2,3,5,6,7,8,10,11,...,99912,99914,99915,99918,99920,99922,99923,99926,99928,99930
id,3edeb34cf93f490ff760af85,c98febd50dad3cc0ffc86085,ade01e13912a46a99134cc75,ab3e6941c11304c1519aef75,252452a91c944a22c276d995,364f7659a4e9ba6c53e35495,c3d87065e4dbc080f400bb85,22e6a518d53f490f6ce7fd85,c6bd550b7f1304c1f7fda485,117b885a3cf9554042ae2285,...,97ee729c6b6b083a05e14995,3e37e6d8bc70bf66f2ca7495,8d403112ca9e2db5ce63d885,2ccac518d420532f256f4595,60ea05f603ff162bd4260195,2460206aa90c63db4475fe85,36f2465b7a5402fcdb8dd595,3cd221a6921fcb56e2dfcd85,40756707daff162bfe68d195,3cddf37632e395c29d764095
fields,"[{'field': {'name': 'Женский гардероб', 'id': ...","[{'field': {'name': 'Детский гардероб', 'id': ...","[{'field': {'name': 'Детский гардероб', 'id': ...","[{'field': {'name': 'Детский гардероб', 'id': ...","[{'field': {'name': 'Детский гардероб', 'id': ...","[{'field': {'name': 'Женский гардероб', 'id': ...","[{'field': {'name': 'Компьютерная техника', 'i...","[{'field': {'name': 'Для дома и дачи', 'id': 4...","[{'field': {'name': 'Детские товары', 'id': 3,...","[{'field': {'name': 'Бытовая техника', 'id': 2...",...,"[{'field': {'name': 'Бытовая техника', 'id': 2...","[{'field': {'name': 'Телефоны и планшеты', 'id...","[{'field': {'name': 'Женский гардероб', 'id': ...","[{'field': {'name': 'Женский гардероб', 'id': ...","[{'field': {'name': 'Недвижимость', 'id': 20, ...","[{'field': {'name': 'Мужской гардероб', 'id': ...","[{'field': {'name': 'Для дома и дачи', 'id': 4...","[{'field': {'name': 'Компьютерная техника', 'i...","[{'field': {'name': 'Красота и здоровье', 'id'...","[{'field': {'name': 'Телефоны и планшеты', 'id..."


In [38]:
train_indp.T

,0,1,2,3,4,5,6,7,8,9,...,1748880,1748881,1748882,1748883,1748884,1748885,1748886,1748887,1748888,1748889
Unnamed: 0,0,1,2,3,5,6,7,8,10,11,...,99912,99914,99915,99918,99920,99922,99923,99926,99928,99930
id,3edeb34cf93f490ff760af85,c98febd50dad3cc0ffc86085,ade01e13912a46a99134cc75,ab3e6941c11304c1519aef75,252452a91c944a22c276d995,364f7659a4e9ba6c53e35495,c3d87065e4dbc080f400bb85,22e6a518d53f490f6ce7fd85,c6bd550b7f1304c1f7fda485,117b885a3cf9554042ae2285,...,97ee729c6b6b083a05e14995,3e37e6d8bc70bf66f2ca7495,8d403112ca9e2db5ce63d885,2ccac518d420532f256f4595,60ea05f603ff162bd4260195,2460206aa90c63db4475fe85,36f2465b7a5402fcdb8dd595,3cd221a6921fcb56e2dfcd85,40756707daff162bfe68d195,3cddf37632e395c29d764095
name,Сумка DG,Комплект,Пальтишко демисезонное,Attipas,Жилет теплый,"Босоножки, сабо",Жесткий диск для ноутбука Seagate 320gb,Ковер 324 на 240,Кенгуру,Машинка автомат,...,Стиральная машинка индезит,Айфон 6 64,Короткое чёрное платье!,Новый купальник,"Участок, до 6 сот., сельхоз (снт или днп)",Кожаная куртка (лайка),Фикус,Мышка беспроводная,Парфюм,HTS Desire 601 dual sim
description,NaN,8-12 лет,"На девочку 1,5 г,состояние хорошее","Размер-135mm, euro-22.5.Прочная, мягкая, не ск...",NaN,"Новые, текстильные босоножки, 37 размер в разм...",Seagate Модель St320lt020 HDD для ноутбуков ...,"Состояние отличное,без дефектов","Продам кенгуру 350 , ванночка с горкой 250",NaN,...,"В хорошем состояние,требуется только замена по...",Телефон упал разбился экран и не включается пр...,Новое! На спине молния,NaN,"г. Реж, СНТ Металлург-5 . Расстояние от Екате...","Кожаная куртка (лайка). Италия, V. King. Новая...",NaN,Новая беспроводная мышка,Парфюмерная вода 50 мл. В наличии одна. Запако...,На экране трещина. Состояние корпуса на 4. Про...
price,199900,35000,30000,80000,50000,30000,150000,200000,35000,350000,...,200000,500000,35000,130000,25000000,749900,20000,25000,70000,210000


In [39]:
X = train_ics[['subcategory']]
Y = train_indp['price']/100

In [40]:
X_train,X_test, Y_train,  Y_test = train_test_split(X,Y, test_size = 0.1, random_state = 25)

In [41]:
X['name'] = train_indp['name']

/home/rromanov/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [42]:
X.head()

,subcategory,name
0,914,Сумка DG
1,2202,Комплект
2,2204,Пальтишко демисезонное
3,2209,Attipas
4,2204,Жилет теплый


In [43]:
Y_train = Y_train.astype(int)

# Ridge

In [44]:
cv = StratifiedKFold(n_splits=5, random_state=8, shuffle=True)
ridge = RidgeCV(cv=cv)
ridge.fit(X_train, Y_train)

/home/rromanov/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:626: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


RidgeCV(alphas=array([ 0.1,  1. , 10. ]),
    cv=StratifiedKFold(n_splits=5, random_state=8, shuffle=True),
    fit_intercept=True, gcv_mode=None, normalize=False, scoring=None,
    store_cv_values=False)

In [47]:
Y_hat = ridge.predict(X_test)
mean_squared_log_error(sqrt(Y_test),Y_hat)

ValueError: Mean Squared Logarithmic Error cannot be used when targets contain negative values.

# Lasso

In [29]:
lasso = LassoCV(cv=cv)
lasso.fit(X_train, Y_train)

/home/rromanov/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:626: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


LassoCV(alphas=None, copy_X=True,
    cv=StratifiedKFold(n_splits=5, random_state=8, shuffle=True),
    eps=0.001, fit_intercept=True, max_iter=1000, n_alphas=100,
    n_jobs=None, normalize=False, positive=False, precompute='auto',
    random_state=None, selection='cyclic', tol=0.0001, verbose=False)

In [32]:
Y_las_hat = lasso.predict(X_test)
mean_squared_log_error(abs(Y_test),Y_las_hat)

38.30198587151696

# Применяем модель

In [40]:
test_ics = pd.read_csv('test_3_col_2.csv', delimiter='\t', index_col='id')

In [41]:
X_test = test_ics[['subcategory']]
test_ics['price'] = ridge.predict(X_test)

In [46]:
test_ics.head()
result=test_ics[['price']]

In [47]:
result.head()

,price
id,
285ea2e9935ccdeb8378c6a5,238886.142623
adfb73820bbb831257df6e95,194168.320114
783025601c36202f633fc6a5,184442.193719
2f0cd2d2e15dc90afd847f95,284274.732470
5c23a37902855a20172845a5,294895.215316


In [49]:
result.to_csv('result.csv')

# Прочее

In [6]:
def unpack_subway(sub, col):
    if sub == 0 or sub == '' or sub == 'NaN':
        return 'Undefined'
    else:
        return sub[col]

def preprocessing_sample(df):
    
    df['price'] = df['price']/100
    df['price'] = df['price'].astype(int)
     # Раскрываем справочники
    df['subway'].fillna(0, inplace=True)
    df['sub_name'] = df['subway'].map(lambda x: unpack_subway(x,'name'))
    df['sub_distance'] = df['subway'].map(lambda x: unpack_subway(x,'distance')).replace('','Undefined')
    df['lat'] = df['location'].map(lambda x: unpack_subway(x,'latitude'))
    df['long'] = df['location'].map(lambda x: unpack_subway(x,'longitude'))
        
    # Преобразуем boolean
    df.replace([False, True], [0,1], inplace=True)
    
    # Преобразуем даты
    df['date_created'] = pd.to_datetime(df['date_created'],unit='s')
    df['year'] = df['date_created'].map(lambda x: x.year)
    df['month'] = df['date_created'].map(lambda x: x.month)
    df['day'] = df['date_created'].map(lambda x: x.day)
    
    # Удаляем лишнее
    df.drop(['category','subway','location', 'images', 'id', 'fields','description','date_created', 'name', 'lat', 'long'], axis=1, inplace=True)
    # 'fields','description', 'name' посмотреть отдельно
    #df.drop(['lat', 'long'], axis=1, inplace=True) #  костыль
    #Удаляем features на основании анализа ниже
    df.drop(['mortgage_available'], axis=1, inplace=True)
    # OneHot
    df = pd.get_dummies(df, columns=['sub_name', 'sub_distance'])
    
    return df

In [7]:
#model = Pipeline ([
#    ('preprocessing', preprocessing_sample),
#    ('onehot', OneHotEncoder(categorical_features=['category','subcategory','sub_name', 'sub_distance'])),
#])

In [8]:
train_main_f = preprocessing_sample(train_sample)

In [22]:
train_main_f.head()

,can_buy,can_promote,contacts_visible,delivery_available,payment_available,price,subcategory,year,month,day,...,sub_name_Площадь Габдуллы Тукая,sub_name_Победа,sub_name_Приморская,sub_name_Тропарево,sub_name_Улица Дыбенко,sub_name_Уральская,sub_distance_Undefined,sub_distance_Более 15 мин. пешком,sub_distance_До 10-15 мин. пешком,sub_distance_На транспорте
0,0,0,1,0,1,1999,914,2017,4,21,...,0,0,0,0,0,0,1,0,0,0
1,0,0,1,0,0,350,2202,2016,10,18,...,0,0,0,0,0,0,1,0,0,0
2,0,0,1,0,0,300,2204,2016,9,4,...,0,0,0,0,0,0,1,0,0,0
3,0,0,1,0,1,800,2209,2016,10,12,...,0,0,0,0,0,0,1,0,0,0
5,0,0,1,0,1,500,2204,2017,8,23,...,0,0,0,0,0,0,1,0,0,0


In [10]:
Y = train_main_f['price']
X = train_main_f.drop(['price'], axis=1)

In [11]:
X_train,X_test, Y_train,  Y_test = train_test_split(X,Y, test_size = 0.3, random_state = 25)

In [12]:
Y_train.shape

(70000,)

   # Check features

In [13]:
def plotCoefficients(model):
    """
    Отрисовывает отсортированные по абсолютному значению коэффициенты модели
    """
    
    coefs = pd.DataFrame(model.coef_, X_lr.columns)
    coefs.columns = ["coef"]
    coefs["abs"] = coefs.coef.apply(np.abs)
    coefs = coefs.sort_values(by="abs", ascending=False).drop(["abs"], axis=1)
    
    plt.figure(figsize=(15, 7))
    coefs.coef[:5].plot(kind='bar')
    plt.grid(True, axis='y')
    plt.hlines(y=0, xmin=0, xmax=len(coefs), linestyles='dashed');

In [14]:
def plotCoefficientsReturn(model):
    """
    Отрисовывает отсортированные по абсолютному значению коэффициенты модели
    """
    
    coefs = pd.DataFrame(model.coef_, X_lr.columns)
    coefs.columns = ["coef"]
    coefs["abs"] = coefs.coef.apply(np.abs)
    coefs = coefs.sort_values(by="abs", ascending=False).drop(["abs"], axis=1)
    
    return coefs[:5].index.values

In [17]:
features = []

for i in subcategoryes:
    Y_lr = train_main_f[train_main_f['subcategory']==i]['price']
    X_lr = train_main_f[train_main_f['subcategory']==i].drop(['price','subcategory'], axis=1)
    lr = LinearRegression()
    lr.fit(X_lr,Y_lr)
    feat = plotCoefficientsReturn(lr)
    for f in feat:
        if f not in features:
            features.append(f)
features

['payment_available',
 'year',
 'delivery_available',
 'month',
 'long',
 'lat',
 'day',
 'can_promote',
 'sub_name_Минская',
 'sub_name_Площадь Габдуллы Тукая',
 'sub_name_Undefined',
 'sub_name_Бунинская Аллея',
 'sub_distance_До 10-15 мин. пешком',
 'sub_name_ВДНХ',
 'sub_distance_Более 15 мин. пешком',
 'sub_distance_Undefined',
 'sub_name_Уральская',
 'contacts_visible',
 'sub_name_Академическая',
 'sub_name_Козья Слобода',
 'sub_name_Победа',
 'sub_name_Тропарево',
 'can_buy']

# Base model (Random Forest)

# Не подошло

In [ ]:
f,ax = plt.subplots(figsize = (21,21))
sns.heatmap(train_main_f.corr(), ax=ax, annot=True)

In [ ]:
import math

#A function to calculate Root Mean Squared Logarithmic Error (RMSLE)
def rmsle(y, y_pred):
	assert len(y) == len(y_pred)
	terms_to_sum = [(math.log(y_pred[i] + 1) - math.log(y[i] + 1)) ** 2.0 for i,pred in enumerate(y_pred)]
	return (sum(terms_to_sum) * (1.0/len(y))) ** 0.5